In [1]:
import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

/root/miniconda3/envs/cuda117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor([   1,    1,    0, 9178,   32,   47]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<pad><pad><s>how are you')

In [2]:
from transformers import AutoModelForCausalLM

model_actor = AutoModelForCausalLM.from_pretrained('model/rlhf',
                                                   torch_dtype=torch.float16,
                                                   device_map='cuda')

In [3]:
import json

with open('dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=8)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(8,
 {'prompt': 'Human: context= CREATE TABLE table_2102898_1 (viewers__in_millions_ VARCHAR, broadcast_date VARCHAR) question= On broadcast date is 28march1970, how many people tuned in? Assistant:',
  'chosen': 'SELECT viewers__in_millions_ FROM table_2102898_1 WHERE broadcast_date = "28March1970"',
  'rejected': '',
  'response': 'SELECT viewers__in_millions_ FROM table_2102898_1 WHERE broadcast_date = "28March1970"'})

In [4]:
for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'])
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = model_actor.generate(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    max_length=256,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id)

    generate = generate[0, 128:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select count(VIEWERS__IN_ILLIONS_) from TABLE_2102898_1 where PROD_DATE = "28march1970"</s>
SELECT viewers__in_millions_ FROM table_2102898_1 WHERE broadcast_date = "28March1970"
select count(WON) from TABLE_15467476_3 where TRIBS_AGAINST = "49"</s>
SELECT won FROM table_15467476_3 WHERE tries_against = "49"
select sum(WEEK) from TABLE_NAME_47 where ATTENDANCE = 72 offset 985</s>
SELECT MAX(week) FROM table_name_47 WHERE attendance = "72,855"
select SCORE from TABLE_22893781_6 where LOCATION_ATTENDANCE = "mADISON square garden 18,828"</s>
SELECT score FROM table_22893781_6 WHERE location_attendance = "Madison Square Garden 18,828"
select FAS from TABLE_NAME_12 where SEASON = 2009–2010 2009–2010 2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009_2009
SELECT fixtures_sec F